In [106]:
# Have to install janitor package to run. Not included in anaconda.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import janitor

In [107]:
exec(open("../header.py").read())

# Import raw data

In [108]:
sample = False
header = Header(sample)

In [109]:
raw_crsp = pd.read_csv(header.clean_root("crsp.csv"), parse_dates = ['date'])
raw_rs = pd.read_csv(header.clean_root("russell.csv"), parse_dates = ['date'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,3,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [110]:
raw_crsp.head(2)

,permno,date,siccd,ncusip,ticker,comnam,permco,cusip,prc,vol,...,bid,ask,shrout,numtrd,ewretd,mktcap,year,month,yrmo,ranks
0,10001,1996-01-02,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,-9.125,0.0,...,8.75,9.5,2281.0,0.0,0.010979,-20814.125,1996,1,199601,8182.0
1,10001,1996-01-03,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,9.500,119.0,...,8.75,9.5,2281.0,1.0,0.005154,21669.500,1996,1,199601,6429.0


In [111]:
raw_rs.head(2)

,date,cusip,ticker,mktvalue,shares,russell1000,russell2000,name,r1000_wt,r2000_wt,year,month,yrmo
0,1996-01-31,000361105,AIR,NaN,15959,0,1,AAR CORP,0.0,0.0599,1996,1,199601
1,1996-01-31,000752105,ABCR,NaN,7983,0,1,ABC RAIL PRODUCTS CO,0.0,0.0304,1996,1,199601


# Problems with CUSIP
* Example: Citigroup
* Link: https://wrds-www.wharton.upenn.edu/pages/support/support-articles/mergent-fisd/linking-mergent-compustat-or-crsp/

In [112]:
print(f"Russell CUSIP for Citigroup: {raw_rs.loc[lambda x:(x.ticker == 'C'),'cusip'].unique()}")
print(f"CRSP CUSIP for Citigroup: {raw_crsp.loc[lambda x:(x.ticker == 'C'),'cusip'].unique()}")
print(f"CRSP NCUSIP for Citigroup: {raw_crsp.loc[lambda x:(x.ticker == 'C'),'ncusip'].unique()}")

Russell CUSIP for Citigroup: ['171196108' '172967101']
CRSP CUSIP for Citigroup: ['17119610' '17296742']
CRSP NCUSIP for Citigroup: ['17119610' '17296710']


In [113]:
raw_crsp.loc[lambda x:(x.ticker == 'C')&(x.date >= '1998-11-12')&(x.date <= '1998-12-8'),['date','ncusip','cusip']]

,date,ncusip,cusip
721132,1998-11-12,17119610,17119610
721133,1998-11-13,17119610,17119610
5977703,1998-12-04,17296710,17296742
5977704,1998-12-07,17296710,17296742
5977705,1998-12-08,17296710,17296742


In [114]:
raw_rs.loc[lambda x:(x.ticker == 'C')&(x.date >= '1998-08-01')&(x.date <= '1999-02-01'),['date','cusip']]

,date,cusip
90868,1998-08-31,171196108
93828,1998-09-30,171196108
96765,1998-10-30,171196108
102601,1998-12-31,172967101
105494,1999-01-29,172967101


# Pre-merge cleaning

In [115]:
rs = raw_rs\
    .drop(['date', 'mktvalue', 'shares', 'year', 'month'], axis = 1)

In [116]:
crsp = raw_crsp\
    .drop(['year','month','cusip'], axis = 1)

# Join on month and ticker instead

In [117]:
crsp_join = crsp\
    .merge(rs, how = "inner", on = ["yrmo", "ticker"])

In [118]:
crsp_join

,permno,date,siccd,ncusip,ticker,comnam,permco,prc,vol,ret,...,ewretd,mktcap,yrmo,ranks,cusip,russell1000,russell2000,name,r1000_wt,r2000_wt
0,10002,2003-06-02,6020,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,7954,15.39,6675.0,-0.010289,...,0.006590,134523.99,200306,3695.0,05978R107,0,1,BANCTRUST FINANCIAL GRP,0.0,0.000150
1,10002,2003-06-03,6020,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,7954,15.17,9411.0,-0.014295,...,0.003286,132600.97,200306,3714.0,05978R107,0,1,BANCTRUST FINANCIAL GRP,0.0,0.000150
2,10002,2003-06-04,6020,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,7954,15.42,12581.0,0.01648,...,0.016344,134786.22,200306,3714.0,05978R107,0,1,BANCTRUST FINANCIAL GRP,0.0,0.000150
3,10002,2003-06-05,6020,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,7954,15.30,5700.0,-0.007782,...,0.010643,133737.30,200306,3724.0,05978R107,0,1,BANCTRUST FINANCIAL GRP,0.0,0.000150
4,10002,2003-06-06,6020,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,7954,14.90,12900.0,-0.026144,...,0.000103,130240.90,200306,3766.0,05978R107,0,1,BANCTRUST FINANCIAL GRP,0.0,0.000150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8228006,93105,2006-12-22,6022.0,92925110,WHI,W HOLDING CO INC,7427,6.05,247800.0,0.0,...,-0.000630,995146.35,200612,2045.0,929251106,0,1,W HOLDING CO INC,0.0,0.000476
8228007,93105,2006-12-26,6022.0,92925110,WHI,W HOLDING CO INC,7427,6.06,290700.0,0.001653,...,0.004164,996791.22,200612,2052.0,929251106,0,1,W HOLDING CO INC,0.0,0.000476
8228008,93105,2006-12-27,6022.0,92925110,WHI,W HOLDING CO INC,7427,6.22,354300.0,0.029015,...,0.008709,1023109.14,200612,2032.0,929251106,0,1,W HOLDING CO INC,0.0,0.000476
8228009,93105,2006-12-28,6022.0,92925110,WHI,W HOLDING CO INC,7427,6.16,340800.0,-0.009646,...,-0.000452,1013239.92,200612,2042.0,929251106,0,1,W HOLDING CO INC,0.0,0.000476


In [122]:
print(f"CRSP // Original: {crsp.shape[0]}, Merged: {crsp_join.shape[0]}, Ratio: {crsp_join.shape[0]/crsp.shape[0]:.2f}")
print(f"Russell // Original: {rs.shape[0]}")
print(f"Unique  companies // Pre: {len(crsp.ticker.unique())}, Post: {len(crsp_join.ticker.unique())}")

CRSP // Original: 21984131, Merged: 8228011, Ratio: 0.37
Russell // Original: 386154
Unique  companies // Pre: 16493, Post: 7331


# Save cleaned datsets

In [120]:
crsp_join.to_csv(header.clean_root("crsp_russ.csv"), index = False)